In [1]:
import os
import pandas as pd

In [2]:
# here I set the path of data set using os.getcwd()
path =  os.getcwd()+"\\data_set"
path

'C:\\Users\\abhis\\AnacondaProjects\\Speech_Classification\\data_set'

In [38]:
# intitalize the empty data frame
data = pd.DataFrame({'text':[],'class':[]})

In [39]:
# call the function to build the data set
data=data.append(build_data_frame(path))

C:\Users\abhis\AnacondaProjects\Speech_Classification\data_set
856


In [26]:
# function for reading the data files
def build_data_frame(path):
    rows = []
    index = []
    classification =[]
    for file_name, text, classification in read_files(path):
        rows.append({'text':text, 'class': classification})
        index.append(file_name)
        
    data_frame = pd.DataFrame(rows,index=index)
    return data_frame

In [27]:
def read_files(path):
    newline=''
    for root, dir_names, file_names in os.walk(path):
        print(root)
        print(len(file_names))
        for file_name in file_names:
            file_path = os.path.join(root,file_name)
            if(os.path.isfile(file_path)):
                #print(file_name)
                if("D" in file_name):
                    label="D"
                elif("R" in file_name):
                    label="R"
                else:
                    label="X"
                lines = []
                f = open(file_path)
                for line in f:
                    lines.append(line.rstrip("\n"))
                f.close()
                content=newline.join(lines)
                yield file_name, content, label

In [40]:
is_republican = data["class"]=="R"
len(data[is_republican])

466

In [41]:
is_democrate=data["class"]=="D"
len(data[is_democrate])

390

In [42]:
data[data["class"]=="X"]

,class,text


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 858 entries, 048_400027_0297016_RMY.txt to 599_400328_2990030_ROY.txt
Data columns (total 2 columns):
class    858 non-null object
text     858 non-null object
dtypes: object(2)
memory usage: 20.1+ KB


In [36]:
data.columns

Index(['class', 'text'], dtype='object')